# **Reinforcement Learning: Taller 4**
## Estudiantes: Juan Pablo Reyes Fajardo y Santiago Rodríguez Ávila 

In [2]:
!pip install gym

In [20]:
import gym
import numpy as np

In [41]:
env = gym.make('CartPole-v1',render_mode="human")
env.reset()

(array([ 0.04134186,  0.00070903,  0.00466769, -0.01977195], dtype=float32),
 {})

In [44]:
env.reset()
for i in range(100):
    action = env.action_space.sample()
    env.step(action)
    env.render()

In [45]:
env.close()

In [62]:
env = gym.make('CartPole-v1')
# Initialize Q-table
Q = np.zeros([env.observation_space.shape[0], env.action_space.n])
print(Q)
# Set learning parameters
alpha = 0.8
gamma = 0.95
epsilon = 0.1

# Initialize variables
num_episodes = 5000
max_steps_per_episode = 200
rewards = []

# Q-learning algorithm
for i in range(num_episodes):
    state = env.reset()
    episode_reward = 0
    for j in range(max_steps_per_episode):
        # Choose action greedily with noise
        if np.random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()
        else:
            print(state[0],"-"*10)
            action = np.argmax(Q[state[0], :])
        # Get new state and reward from environment
        new_state, reward, done, _,_= env.step(action)
        print(new_state)
        # Update Q-table
        print(Q.shape,"*"*8)
        Q[state, action] = (1 - alpha) * Q[state, action] + alpha * (reward + gamma * np.max(Q[new_state, :]))
        episode_reward += reward
        state = new_state
        if done:
            break
    rewards.append(episode_reward)

[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
[-0.04711828 -0.18638623 -0.04193974  0.2802773 ]
(4, 2) ********


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [27]:
prueba = np.zeros([4,2])
prueba[0,1]= 2
prueba[1,0]= 4
prueba[2,0]= 1
prueba[3,1]= 8

state = env.reset()

print(state[0])
#print(env.observation_space)
#env.observation_space.low[2]
#prueba[state,0]
#print(prueba)
#print(prueba[1,:])
#np.argmax(prueba[0,:])

[-0.02061262 -0.00248625  0.00048967 -0.04600552]
